Ejemplo de estrategia con la libreria ib_insync
===============================================

* market:       NQ
* timeframe:    2 min
* reglas de entrada:    two_bars_up   and   close > bollinger_up_band (10, 1.5) 
* reglas de salida:     close < min(6)   or   close < close[2] - 1.5 * atr(14)

In [36]:
# importar las librerias necesarias
# !pip install ib_insync
import pandas as pd
import numpy as np
import time
from ib_insync import *  # https://ib-insync.readthedocs.io/readme.html
import asyncio

In [37]:
# 1. CONEXION A IB (TWS debe estar arrancada previamente y configurada para API)
# https://interactivebrokers.github.io/tws-api/initial_setup.html

ib = IB()
util.startLoop() # comentar esta linea si no es un notebook
ib.connect('127.0.0.1', 7497, clientId= 1) # para cuenta demo, cliente 1
ib.sleep(1) # para asegurar la conexion

True

In [38]:
# 2. SE DEFINE EL ACTIVO O MERCADO
# (https://interactivebrokers.github.io/tws-api/basic_contracts.html)

contract = ContFuture('NQ', 'GLOBEX')
size = 1 # nº de contratos
ib.qualifyContracts(contract)

[ContFuture(conId=497954599, symbol='NQ', lastTradeDateOrContractMonth='20220916', multiplier='20', exchange='GLOBEX', currency='USD', localSymbol='NQU2', tradingClass='NQ')]

In [39]:
# 3. SE DEFINE EL TIMEFRAME Y EL HISTORICO A DESCARGAR
# https://interactivebrokers.github.io/tws-api/historical_bars.html

data = ib.reqHistoricalData(
    contract,
    endDateTime='', # hasta el instante actual
    durationStr='1 D', # la duracion del historico requerido
    barSizeSetting='2 mins', # timeframe
    whatToShow='TRADES', 
    useRTH=False, # 24 h
    keepUpToDate=True, # actualizacion continua
    formatDate=1)  # tipo de datos segun la cuenta


In [40]:
# definimos 2 df: df_in y df_out donde se ira almacenando informacion de la estrategia para monitorizacion y posterior analisis
# df_in: ohlcv + indicadores + condiciones + señales de entrada/salida
# df_out: ordenes y posiciones

df_in = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'three_bars_up', 'atr_14', 'min_6', 'in_condition_1', 'in_condition_2', 'out_condition_1', 'out_condition_2', 'signal_in', 'signal_out'])
df_out = pd.DataFrame(columns=['date', 'current_pos', 'order_sent'])

# los guardamos en pickle
df_in.to_pickle('df_in.pkl')
df_out.to_pickle('df_out.pkl')

In [41]:
# 4. SE DEFINEN LAS ACCIONES A REALIZAR EN CADA NUEVA BARRA

def on_new_bar(bars: BarDataList, has_new_bar: bool): 
    if has_new_bar: 
        
        # obtenemos los datos ohlcv
        df = util.df(data)
        df = df.drop(['volume', 'average', 'barCount'], axis=1) # eliminamos las columnas volume, average y barCount que no queeemos utilizar

        # anadimos los indicadores de la estrategia
        df['two_bars_up'] = (df['close'] > df['open']) & (df['close'].shift(1) > df['open'].shift(1))
        df['atr_14'] = np.max(pd.concat([df['high'] - df['low'], np.abs(df['high'] - df['close'].shift()), np.abs(df['low'] - df['close'].shift())], axis=1), axis=1).rolling(14).sum()/14
        df['min_6'] = df['close'].rolling(window=6, min_periods=1).min()
        df['bb_upband'] = df['close'].rolling(10).mean() + (df['close'].rolling(10).std() * 1.5)
        
        # anadimos las condiciones o reglas de la estrategia
        df['in_condition_1'] = df['two_bars_up']
        df['in_condition_2'] = df['close'] > df['bb_upband']
        df['out_condition_1'] = df['close'] < df['min_6']
        df['out_condition_2'] = df['close'] < (df['close'].shift(2) - 1.5 * df['atr_14'])

        # generamos las senales de la estrategia
        #  1 --> BUY
        # -1 --> SELL
        df["signal_in"] = np.where(df['in_condition_1'] & df['in_condition_2'], 1, 0)
        df["signal_out"] = np.where(df['out_condition_1'] | df['out_condition_2'], -1, 0)

        df.to_pickle('df_in.pkl')
        
        # leemos la posicion del broker 
        df_pos = pd.DataFrame(ib.positions())
        time.sleep(1) 
        
        if df_pos.empty:
            current_pos = 0
        else:
            current_pos = df_pos.loc[0, 'position'] # (no valido si mas de un activo)

        # enviamos las ordenes a mercado segun la estrategia:
        # lanzaremos orden a mercado de venta si en la penultima barra, signal_out es -1 y habia posicion abierta 
        # (la barra acual aun no se ha cerrado, por lo que no tiene datos validos)
        if ((df['signal_in'].iloc[-2] == 0 ) & (df['signal_out'].iloc[-2] == -1) & (current_pos > 0)):
            order = MarketOrder('SELL', size)
            trade = ib.placeOrder(contract, order)
            order_sent = 'SELL'
        else:
            order_sent = 'None'

        # lanzaremos orden a mercado de compra si en la penultima barra, signal_in es 1 y no habia posicion abierta
        if ((df['signal_in'].iloc[-2] == 1 ) & (df['signal_out'].iloc[-2] == 0) & (current_pos == 0)):
            order = MarketOrder('BUY', size)
            trade = ib.placeOrder(contract, order)
            order_sent = 'BUY'
    
        #  actualizamos df_out
        df_out = pd.read_pickle('df_out.pkl')
        df_out = df_out.append({'date': df['date'].iloc[-1], 'current_pos': current_pos, 'order_sent': order_sent}, ignore_index=True)
        df_out.to_pickle('df_out.pkl')


In [42]:
# ejecutamos de forma continua

data.updateEvent += on_new_bar
ib.run()

In [48]:
# MONITORIZAMOS LOS RESULTADOS

df_in = pd.read_pickle('df_in.pkl')
df_out = pd.read_pickle('df_out.pkl')
df = pd.merge(df_in, df_out, on='date')
df # monitorizamos las operaciones realizadas

,date,open,high,low,close,two_bars_up,atr_14,min_6,bb_upband,in_condition_1,in_condition_2,out_condition_1,out_condition_2,signal_in,signal_out,current_pos,order_sent
0,2022-09-02 08:04:00,12249.00,12253.00,12244.25,12248.50,False,7.660714,12248.50,12273.854255,False,False,False,False,0,0,1.0,SELL
1,2022-09-02 08:06:00,12248.25,12258.50,12244.25,12258.25,False,8.196429,12248.50,12273.503618,False,False,False,False,0,0,0.0,None
2,2022-09-02 08:08:00,12258.50,12259.50,12249.25,12253.75,False,8.642857,12248.50,12273.244627,False,False,False,False,0,0,0.0,None
3,2022-09-02 08:10:00,12253.25,12258.50,12253.25,12256.75,False,8.642857,12248.50,12271.626791,False,False,False,False,0,0,0.0,None
4,2022-09-02 08:12:00,12256.75,12264.00,12256.75,12261.00,True,8.428571,12248.50,12269.743232,True,False,False,False,0,0,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,2022-09-02 13:36:00,12260.00,12272.25,12257.75,12271.25,True,8.660714,12251.25,12265.940663,True,True,False,False,1,0,0.0,None
167,2022-09-02 13:38:00,12271.50,12275.25,12270.00,12275.25,True,7.500000,12251.25,12271.048122,True,True,False,False,1,0,0.0,BUY
168,2022-09-02 13:40:00,12275.50,12277.75,12272.25,12275.00,False,7.375000,12253.00,12275.044075,False,False,False,False,0,0,1.0,None
169,2022-09-02 13:42:00,12274.50,12281.50,12274.50,12278.75,False,7.285714,12257.75,12279.038553,False,False,False,False,0,0,1.0,None


In [44]:
#ib.disconnect()

In [45]:
#df.to_csv('resultados.csv')